In [144]:
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
%run ./2-ImplementationFactor.ipynb
%run ./3-ImplementationPGM.ipynb
%run ./4-ImplementationBP.ipynb

./img/3/graph.html
./img/3/graph.html


In [145]:
def delta_funct(S_ki, t_i):
    if (t_i == np.min(S_ki)):
        return 1
    return 0

In [146]:
def R_i(t_i, r_i, lambda_i):
    if (t_i > r_i):
        return 0
    result = lambda_i*np.exp(-lambda_i*(r_i - t_i))
    return result

In [147]:
def S_ij(s_ij, t_i, r_i, lambda_i, time_range, contact_range_ij):
    if (t_i <= s_ij < r_i):
            temp = np.where((contact_range_ij < s_ij) & (contact_range_ij >= t_i))
            return (lambda_i)*(1-lambda_i)**(len(temp))
    elif (s_ij ==9):
        index_larger_than_t_i = np.where(contact_range_ij >= t_i)
        return (1-lambda_i)**(len(index_larger_than_t_i))
    else:
        return 0

In [148]:
#debug function
def Find_Zero(delt, R, S):
    if (delt == 0):
        print("delt = 0")
    if (R == 0):
        print("R = 0")
    if (S == 0):
        print("S = 0")

def phi(t_i,r_i, s, type):
    delt = delta_funct([s[0]],t_i)
    R = R_i(t_i, r_i, 0.3)
    S = S_ij(s[0], t_i, r_i, 0.3, "nothing", np.array([0,6,9]))
    #Find_Zero(delt, R, S)
    return delt*R*S


In [149]:
#phi(t_i,r_i,(s_ij,s_ji), type)

In [150]:
phi(0, 1, (0,0), 0)

0.046671547902948224

In [151]:
#inference using BP

In [152]:
variables_i = ["t_i","r_i","s_ij:s_ji"]
variables_j = ["t_j", "r_j","s_ij:s_ji"]

In [153]:
shape = (10,10,10,10)
matrix_phi_i = np.zeros(shape=(10,10,10))
matrix_phi_j = np.zeros(shape=(10,10,10))
for index in np.ndindex(shape):
    matrix_phi_i[index[0], index[1], index[2]] = phi(index[0], index[1], [index[2]],0)
    matrix_phi_j[index[0], index[1], index[2]] = phi(index[0], index[1], [index[2]],1)

In [154]:
phi_i = factor(variables_i, matrix_phi_i)
phi_j = factor(variables_j, matrix_phi_j)

In [155]:
graph = factor_graph()
graph.add_factor_node("phi_i", phi_i)
graph.add_factor_node("phi_j", phi_j)

In [156]:
bp = belief_propagation(graph)
bp_result = bp.belief("t_i").get_distribution()

In [157]:
bp_result

array([0.1394739 , 0.13253123, 0.1234407 , 0.11168195, 0.09674255,
       0.07827696, 0.05644955, 0.03263148, 0.01076787, 0.21800382])

In [158]:
plot_factor_graph(graph)

./img/3/graph.html


In [159]:
#exact calculation of marginals

In [160]:
t_i_array = []
for t_i in range(10):
    sum = 0
    for r_i in range(10):
        for t_j in range(10):
            for r_j in range(10):
                for s_ij in range(10):
                    for s_ji in range(10):
                        sum += phi_i.get_distribution()[t_i, r_i, s_ij]*phi_j.get_distribution()[t_j, r_j, s_ji]
    t_i_array.append(sum)

In [161]:
sum_Z = np.sum(t_i_array)

In [162]:
t_i_array

[0.22685184984790147,
 0.22113371932078385,
 0.21341505046588488,
 0.20299593732933702,
 0.18893160569483713,
 0.16994674376533764,
 0.14431986067918845,
 0.10972718683462956,
 0.06303196136794771,
 0.28361416070438195]

In [163]:
marginal_t_i = t_i_array/sum_Z
marginal_t_i

array([0.12437271, 0.12123771, 0.11700591, 0.11129358, 0.10358274,
       0.09317419, 0.07912412, 0.0601585 , 0.0345576 , 0.15549294])

In [164]:
bp_result

array([0.1394739 , 0.13253123, 0.1234407 , 0.11168195, 0.09674255,
       0.07827696, 0.05644955, 0.03263148, 0.01076787, 0.21800382])

In [165]:
factor_i = factor(["t_i", "t_j"], np.array([[0.5, 0.6], [0.7, 0.8]]))
factor_j = factor(["t_j"], np.array([0.5, 0.6]))

graph = factor_graph()
#graph.add_factor_node("factor_i", factor_i)
graph.add_factor_node("factor_j", factor_j)

bp = belief_propagation(graph)
bp_result = bp.belief("t_j").get_distribution()

In [166]:
bp_result

array([0.45454545, 0.54545455])

In [167]:
#exact calculation of marginals
exact = []
matrix =  np.array([[0.5, 0.6], [0.7, 0.8]])
for t_j in range(2):
        sum = 0
        sum += factor_i.get_distribution()[0,t_j]
        exact.append(sum)
sum_Z = np.sum(exact)
exact

[0.5, 0.6]

In [168]:
fac = factor_reduction(factor_i, "t_i", 0)

In [169]:
np.array(exact)/sum_Z

array([0.45454545, 0.54545455])